In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Load in CSV data files - Regular and Playoffs Games

In [ ]:
# Load the regular season data
file_path = Path('.../Resources')

reg_df = pd.read_csv(f"{file_path}/raptors_regulars.csv")
